In [1]:
import pandas as pd

### Read Compustat Data

In [5]:
compustat_df = pd.read_pickle('../get_data/compustat_data.pkl')
compustat_variables = pd.read_csv('../get_data/compustat_variables.csv')

In [3]:
compustat_variables = compustat_variables['Name']
general_items = ['gvkey','datadate','datacqtr', 'cusip','tic', 'gsector'] + ['mkvaltq']

In [4]:
compustat_df = compustat_df[general_items + list(compustat_variables)]
compustat_df.head()

,gvkey,datadate,datacqtr,cusip,tic,gsector,mkvaltq,acchgq,acomincq,acoq,...,txpq,txtq,wcapq,xaccq,xintq,xiq,xoprq,xrdq,xsgaq,prccq
0,1004,1985-08-31,1985Q3,000361105,AIR,20,NaN,NaN,NaN,6.138,...,NaN,1.96,58.163,NaN,0.735,0.0,47.735,NaN,7.475,22.625
1,1004,1985-11-30,1985Q4,000361105,AIR,20,NaN,NaN,NaN,6.665,...,NaN,2.33,57.965,NaN,0.723,0.0,54.194,NaN,8.243,24.000
2,1004,1986-02-28,1986Q1,000361105,AIR,20,NaN,NaN,NaN,7.025,...,NaN,2.35,59.787,NaN,0.751,0.0,57.559,NaN,8.746,20.375
3,1004,1986-05-31,1986Q2,000361105,AIR,20,NaN,NaN,NaN,6.162,...,1.054,2.46,70.657,NaN,0.799,0.0,61.242,NaN,10.526,23.625
4,1004,1986-08-31,1986Q3,000361105,AIR,20,NaN,NaN,NaN,8.478,...,NaN,2.53,88.918,NaN,1.100,0.0,58.741,NaN,8.962,23.500


In [5]:
print(f'Number of Companies: {compustat_df["gvkey"].nunique()}')
print(f'Number of Rows: {len(compustat_df)}')
print(f'Number of Columns: {len(compustat_df.columns)}')

Number of Companies: 5664
Number of Rows: 359988
Number of Columns: 81


### Read Earnings Data

In [6]:
earnings_df = pd.read_csv('../get_data/earnings_data.csv')
earnings_df = earnings_df[earnings_df['FPI']==6].reset_index(drop=True)

In [7]:
earnings_df = earnings_df.sort_values(by='STATPERS', ascending=True)
earnings_df = earnings_df.groupby(['cusip','FPEDATS']).head(1)
earnings_df = earnings_df.sort_values(by=['cusip','FPEDATS']).reset_index(drop=True)

In [8]:
earnings_df = earnings_df[['cusip','FPEDATS','STATPERS', 'ACTUAL','MEANEST','MEDEST','ANNDATS_ACT']]

In [9]:
earnings_df.head()

,cusip,FPEDATS,STATPERS,ACTUAL,MEANEST,MEDEST,ANNDATS_ACT
0,00032Q10,2021-09-30,2021-09-16,-1.36,-6.00,-6.00,2021-11-10
1,00032Q10,2021-12-31,2021-11-18,-0.77,-0.95,-0.88,2022-03-17
2,00032Q10,2022-03-31,2022-03-17,-0.66,-0.81,-0.78,2022-05-12
3,00032Q10,2022-06-30,2022-05-19,-0.87,-0.75,-0.73,2022-08-10
4,00032Q10,2022-09-30,2022-08-18,-0.68,-0.74,-0.75,2022-11-09


In [10]:
earnings_df = earnings_df.dropna()

In [11]:
earnings_df.shape

(185626, 7)

### Merge Compustat and Earnings Data

In [12]:
compustat_df['cusip_eight'] = compustat_df['cusip'].str[:-1]

In [13]:
earnings_df['cusip'].nunique(), compustat_df['cusip_eight'].nunique()

(4429, 5664)

In [14]:
merged_df = pd.merge(compustat_df, earnings_df, how='left',left_on=['cusip_eight','datadate'], right_on=['cusip','FPEDATS'])

In [15]:
merged_df = merged_df.drop(columns={'datadate','cusip_eight','cusip_y','FPEDATS'})
merged_df = merged_df.dropna(subset=['ACTUAL','MEANEST','MEDEST'])
merged_df = merged_df.reset_index(drop=True)

In [16]:
merged_df = merged_df.rename(columns={'ACTUAL':'eps_actual', 
                                    'MEANEST':'eps_predicted_mean', 
                                    'MEDEST':'eps_predicted_median',
                                    'ANNDATS_ACT':'announcement_date',
                                    'STATPERS':'analyst_date',
                                    'cusip_x':'cusip'})

In [17]:
merged_df.head()

,gvkey,datacqtr,cusip,tic,gsector,mkvaltq,acchgq,acomincq,acoq,actq,...,xiq,xoprq,xrdq,xsgaq,prccq,analyst_date,eps_actual,eps_predicted_mean,eps_predicted_median,announcement_date
0,1004,1985Q4,000361105,AIR,20,NaN,NaN,NaN,6.665,111.783,...,0.0,54.194,NaN,8.243,24.000,1985-11-14,0.1732,0.13,0.13,1985-12-19
1,1004,1986Q1,000361105,AIR,20,NaN,NaN,NaN,7.025,118.121,...,0.0,57.559,NaN,8.746,20.375,1985-12-19,0.1419,0.13,0.13,1986-03-21
2,1004,1986Q2,000361105,AIR,20,NaN,NaN,NaN,6.162,140.741,...,0.0,61.242,NaN,10.526,23.625,1986-03-20,0.1599,0.15,0.15,1986-07-08
3,1004,1986Q3,000361105,AIR,20,NaN,NaN,NaN,8.478,148.209,...,0.0,58.741,NaN,8.962,23.500,1986-07-17,0.1466,0.16,0.16,1986-09-24
4,1004,1986Q4,000361105,AIR,20,NaN,NaN,NaN,7.880,148.136,...,0.0,66.944,NaN,9.385,23.500,1986-10-16,0.1665,0.18,0.18,1986-12-16


In [18]:
print(f'Number of Companies: {merged_df["gvkey"].nunique()}')
print(f'Number of Rows: {len(merged_df)}')
print(f'Number of Columns: {len(merged_df.columns)}')

Number of Companies: 3165
Number of Rows: 141178
Number of Columns: 85


In [19]:
merged_df.to_pickle('../data/merged_data.pkl')